In [ ]:
pip install gluonnlp

In [ ]:
pip install mxnet

     |████████████████████████████████| 46.9MB 62kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [ ]:
pip install transformers

     |████████████████████████████████| 2.1MB 10.9MB/s 
     |████████████████████████████████| 901kB 41.5MB/s 
     |████████████████████████████████| 3.3MB 43.4MB/s 


In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.7MB/s 
     |████████████████████████████████| 92kB 10.6MB/s 
     |████████████████████████████████| 460kB 35.8MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
pip install wget

  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9681 sha256=f5568627c026387da04f28d23816c9bc92872abb288b4f76c5a79f566cb2d67c
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [ ]:
pip install sentencepiece

     |████████████████████████████████| 1.2MB 10.3MB/s 


In [2]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import BertTokenizer
import tensorflow as tf
from tensorflow import keras
import json
import re
from tqdm import tqdm
from konlpy.tag import Okt
import gluonnlp as nlp
from gluonnlp.data import SentencepieceTokenizer
from transformers import TFGPT2LMHeadModel

# **GTP2를 활용한 한국어 언어 생성 모델**

In [3]:
class GPT2Model(tf.keras.Model):
  def __init__(self, dir_path):
    super(GPT2Model, self).__init__()
    self.gpt2 = TFGPT2LMHeadModel.from_pretrained(dir_path)

  def call(self, inputs):
    return self.gpt2(inputs)[0]

In [ ]:
import wget
import zipfile

wget.download('https://github.com/NLP-kr/tensorflow-ml-nlp-tf2/releases/download/v1.0/gpt_ckpt.zip')
with zipfile.ZipFile('gpt_ckpt.zip') as z:
  z.extractall()

In [4]:
BASE_MODEL_PATH = '/content/gpt_ckpt'
gpt_model = GPT2Model(BASE_MODEL_PATH)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/gpt_ckpt.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [5]:
TOKENIZER_PATH = '/content/gpt_ckpt/gpt2_kor_tokenizer.spiece'
tokenizer = SentencepieceTokenizer(TOKENIZER_PATH)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(TOKENIZER_PATH,
                                               mask_token=None,
                                               sep_token=None,
                                               cls_token=None,
                                               unknown_token='<unk>',
                                               padding_token='<pad>',
                                               bos_token='<s>',
                                               eos_token='</s>')

In [6]:
def generate_sent(seed_word, model, max_step=200, greedy=False, top_k=0, top_p=0.4):
  # 인자로 들어온 문장 생성을 시작할 단어를 문장을 의미하는 변수에 할당하고 토크아니즈한다.
  sent = seed_word
  toked = tokenizer(sent)

  # 문장 생성을 하는 반복문
  # 토크나이즈된 단어를 인덱스로 반환하고 모델에 입력값으로 넣어 출력값을 받는다.
  # 모델의 출력값에 대해서는 문장에서 마지막 단어만 선택하게 한다.
  for _ in range(max_step):
    input_ids = tf.constant([vocab[vocab.bos_token],] + vocab[toked])[None, :] 
    outputs = model(input_ids)[:, -1, :]


    if greedy:
      gen = vocab.to_tokens(tf.argmax(outputs, axis=-1).numpy().tolist()[0])
    else:
      output_logit = tf_top_k_top_p_filtering(outputs[0], top_k=top_k, top_p=top_p)
      gen = vocab.to_tokens(tf.random.categorical(output_logit, 1).numpy().tolist()[0])[0]
    if gen == '</s>':
      break
    sent += gen.replace(' ', ' ')
    toked = tokenizer(sent)

  return sent

# - greedy 파라미터
모델에서 출력을 마쳤다면 출력한 마지막 토큰에서 가장 확률이 높은 단어를 선택하거나 확률이 높은 단어 중에서 확률 분로에 따라 선택할 수 있게 해야 한다.

**greedy값이 True인 경우** tf.argmax로 가장 확률이 높은 단어를 선택하고 텍스트를 반환한다.

**greedy값이 False인 경우** tf_top_k_p_filtering함수를 통해 무작위로 선택할 단어를 필터링한다.
그리고 tf.random.categorical을 통해 무작위로 선택한 단어들의 확률 분포를 토대로 무작위 선택을 하고 텍스트로 변환한다.
[여기서는 top_k 값을 0으로 설정하고 top_p만 조절해서 문장을 생성하는 뉴클러스 샘플링을 활용한다.]

In [99]:
print(generate_sent('이때', gpt_model, greedy=True,top_k=0, top_p=9.5))

이때▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


# **소설 텍스트**

소설 텍스트 데이터 전처리

In [7]:
import nltk

DATA_IN_PATH = '/content/'
TRAIN_DATA_FILE = '셔츠남방.txt'

sents = [s[:-1] for s in open(DATA_IN_PATH + TRAIN_DATA_FILE, 'r', encoding='utf-8').readlines()] #, encoding='utf-8'

In [8]:
csv_test = pd.read_csv('/content/01_원피스_meta_data.csv')
csv_test2 = pd.read_csv('/content/02_스커트_치마_meta_data.csv')
csv_test3 = pd.read_csv('/content/03_자켓_meta_data.csv')
csv_test4 = pd.read_csv('/content/04_니트_스웨터_meta_data.csv')
csv_test5 = pd.read_csv('/content/08_반팔티셔츠_meta_data.csv')

test = csv_test[['detail']]
test2 = csv_test2[['detail']]
test3 = csv_test3[['detail']]
test4 = csv_test4[['detail']]
test5 = csv_test5[['detail']]

df = pd.concat([test, test2, test3, test4, test5])
a = df['detail'].tolist()

sents = str(sents + a)

In [9]:
# 학습데이터는 텍스트를 먼저 문장별로 분리해둔 텍스트데이터이다.

input_data = []
output_data = []

# 언어 생성 모델은 다음 단어를 예측하는 데이터 구조로 구성
# 입력데이터를 tokens[:-1]로 맨 앞에서 맨 뒤 직전 토큰까지만 활용
# 정답데이터를 tokens[1:]로 맨 앞 다음 코든에서 맨 뒤 토큰까지 활용하게 한다.

for s in sents:
  tokens = [vocab[vocab.bos_token],] + vocab[tokenizer(s)] + [vocab[vocab.eos_token],]
  input_data.append(tokens[:-1])
  output_data.append(tokens[1:])

In [11]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding
# pad_sequences함수를 활용하여 패딩
# np.array로 구성된 학습데이터 준비

input_data = pad_sequences(input_data, 200, value=vocab[vocab.padding_token])
output_data = pad_sequences(output_data, 200, value=vocab[vocab.padding_token])

input_data = np.array(input_data, dtype=np.int64)
output_data = np.array(output_data, dtype=np.int64)

# **-** (구) 모델과 (신) 모델의 차이
과거 모델의 약점: 신조어나 줄임말에 대한 예측이 약함
예)RNN

과거의 모델에 반해서 최신모델은 신조어나 줄임말에 대한 상대적인 강함을 보임. 

# **- maxlen() **
maxlen은 metrlx를 형성하는 과정에서 모든 시퀀스 중 가장 긴 길이로 시퀀스 크기를 선정한다. 만약 문장의 길이가 가장 킨 시퀀스에 모자르면 나머지는 '0'으로 채워진다. 

무조건적으로 가장 긴 길이로 시퀀스로 정하는 것이 좋은 것이 아니며(->'0'이 많아지면 학습에 영향을 줌) 50%만 남기는 것은 아까운 일이다.(보통 75%정로로 선정) 

# 소설 텍스트 미세 조정 모델

In [12]:
# loss와 accuracy

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, vocab[vocab.padding_token]))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

def accuracy_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, vocab[vocab.padding_token]))
  mask = tf.expand_dims(tf.cast(mask, dtype=pred.dtype), axis=-1)
  pred *= mask

  acc = train_accuracy(real, pred)

  return tf.reduce_mean(acc)

In [13]:
# gpt_model.compile을 통해 loss나 optimizer, metrics를 설정
gpt_model.compile(loss=loss_function,
                  optimizer=tf.keras.optimizers.Adam(1e-4),
                  metrics=[accuracy_function])

In [ ]:
BATCH_SIZE = 36
NUM_EPOCHS = 10
VALID_SPLIT = 0.2

history = gpt_model.fit(input_data,
                        output_data,
                        batch_size=BATCH_SIZE,
                        epochs=NUM_EPOCHS,
                        validation_split=VALID_SPLIT)  

Epoch 1/10
 4998/28866 [====>.........................] - ETA: 7:33:57 - loss: 0.0228 - accuracy_function: 0.0090

In [ ]:
# config.json과 tf_model.h5파일을 폴더에 저장

DATA_OUT_PATH = "./data_out"
model_name = 'tf2_gp2_finetuned_model'

save_dir_path = os.path.join(DATA_OUT_PATH, model_name)
save_file_path = os.path.join(save_dir_path, 'weights.h5')

if not os.path.exists(save_dir_path):
  os.makedirs(save_dir_path)

gpt_model.gpt2.save_pretrained(save_file_path)
load_gpt_model = GPT2Model(save_file_path)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at ./data_out/tf2_gp2_finetuned_model/weights.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [135]:
generate_sent('가디건, 빨강', gpt_model, greedy=True, top_k=0, top_p=0.5)

'가디건, 빨강,▁파랑,▁파랑,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁'